In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

sys.path.append('..')

from mim.cross_validation import ChronologicalSplit
from mim.experiments.ab_glucose import ABGlucose
import mim.extractors.ab_json as ab_json
import mim.models.ab_nn as ab_nn

In [2]:
specification = {
    "labels": {"target": "label-index+30d-ami+death-30d"},
    "index": {"json_train": "/home/sapfo/andersb/PycharmProjects/Expect/json_data/pontus_glukos/hbg+lund-train.json.gz"},
    "features":{
        "gender",
        "age",
        "bl-Glukos", "bl-TnT", "bl-Krea", "bl-Hb"
    },
}

experiment = ABGlucose.KERAS_LR_BLOOD_BL

In [3]:
development, hold_out = experiment.get_data()

{'gender', 'bl-Glukos', 'age', 'bl-TnT', 'bl-Krea', 'bl-Hb'}
['age', 'bl-Glukos', 'bl-Hb', 'bl-Krea', 'bl-TnT']
['gender']


In [4]:
train, val = next(experiment.cross_validation.split(development))

In [5]:
keras_model = ab_nn.ab_simple_lr(train)

In [6]:
keras_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
categorical (InputLayer)        [(None, 2)]          0                                            
__________________________________________________________________________________________________
numeric (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 7)            0           categorical[0][0]                
                                                                 numeric[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            8           concatenate[0][0]     

In [7]:
len(hold_out)

1191

In [8]:
compile_args = {
    'optimizer': 'sgd',
    'loss': 'binary_crossentropy',
    'metrics': ['accuracy', tf.keras.metrics.AUC()]
}
keras_model.compile(**compile_args)

In [9]:
epochs=experiment.epochs
batch_size=experiment.batch_size

x = train['x'].as_dataset
y = train['y'].as_dataset

random_state = experiment.random_state
np.random.seed(random_state)
tf.random.set_seed(random_state)

v = tf.data.Dataset.zip((val['x'].as_dataset, val['y'].as_dataset)).batch(batch_size).prefetch(3)

keras_model.fit(x=tf.data.Dataset.zip((x, y)).batch(batch_size).prefetch(3), 
                validation_data=v,
                epochs=epochs)

Epoch 1/10
75/75 [==============================] - 1s 17ms/step - loss: 8.6275 - accuracy: 0.8532 - auc: 0.6048 - val_loss: 11.6335 - val_accuracy: 0.9203 - val_auc: 0.5526
Epoch 2/10
75/75 [==============================] - 1s 11ms/step - loss: 6.1761 - accuracy: 0.8745 - auc: 0.6214 - val_loss: 3.9156 - val_accuracy: 0.9262 - val_auc: 0.6140
Epoch 3/10
75/75 [==============================] - 1s 14ms/step - loss: 6.8445 - accuracy: 0.8653 - auc: 0.5937 - val_loss: 2.6919 - val_accuracy: 0.8796 - val_auc: 0.8343
Epoch 4/10
75/75 [==============================] - 1s 13ms/step - loss: 7.4007 - accuracy: 0.8637 - auc: 0.6085 - val_loss: 14.0714 - val_accuracy: 0.9194 - val_auc: 0.5476
Epoch 5/10
75/75 [==============================] - 1s 13ms/step - loss: 6.6530 - accuracy: 0.8800 - auc: 0.6194 - val_loss: 3.7948 - val_accuracy: 0.9254 - val_auc: 0.6147
Epoch 6/10
75/75 [==============================] - 1s 13ms/step - loss: 6.7426 - accuracy: 0.8737 - auc: 0.6167 - val_loss: 2.6065 -